# *ECG Image-to-Signal Reconstruction*

## Import Libraries

In [ ]:
# Step 1: Basic Setup & File Check

import os
import pandas as pd

# Base path
DATA_PATH = '/kaggle/input/physionet-ecg-image-digitization/'

# List all files
for dirname, _, filenames in os.walk(DATA_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Check main CSV files
train_csv = os.path.join(DATA_PATH, 'train.csv')
test_csv = os.path.join(DATA_PATH, 'test.csv')

# Load CSVs
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

# Basic info
print("\n✅ Train shape:", train_df.shape)
print("✅ Test shape:", test_df.shape)

# Preview first few rows
print("\n📄 Train data preview:")
display(train_df.head())

print("\n📄 Test data preview:")
display(test_df.head())


##  Data Overview & Missing Values

In [ ]:
# Basic info about train and test datasets
print("=== TRAIN DATA INFO ===")
train_df.info()
print("\n=== TEST DATA INFO ===")
test_df.info()

# Check missing values
print("\n=== Missing Values (Train) ===")
print(train_df.isnull().sum())

print("\n=== Missing Values (Test) ===")
print(test_df.isnull().sum())

# Basic descriptive stats for numeric columns
print("\n=== Descriptive Statistics (Train) ===")
display(train_df.describe())

# Unique values for object-type columns
print("\n=== Unique Values in Object Columns (Train) ===")
for col in train_df.select_dtypes(include='object').columns:
    print(f"{col}: {train_df[col].nunique()} unique values")


## Data Distribution & Relationships

In [ ]:
# Step 3: Data Distribution & Relationships

import matplotlib.pyplot as plt
import seaborn as sns

# Style setup
sns.set(style="whitegrid")

# 1️⃣ Histogram for fs
plt.figure(figsize=(6,4))
sns.histplot(train_df['fs'], bins=20, kde=True)
plt.title('Distribution of Sampling Frequency (fs)')
plt.xlabel('fs')
plt.ylabel('Count')
plt.show()

# 2️⃣ Histogram for sig_len
plt.figure(figsize=(6,4))
sns.histplot(train_df['sig_len'], bins=20, kde=True, color='orange')
plt.title('Distribution of Signal Length (sig_len)')
plt.xlabel('sig_len')
plt.ylabel('Count')
plt.show()

# 3️⃣ Scatter plot: fs vs sig_len
plt.figure(figsize=(6,4))
sns.scatterplot(data=train_df, x='fs', y='sig_len')
plt.title('fs vs sig_len Relationship')
plt.show()


## Train–Test Comparison & Summary Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Combine 'fs' values from both train and test for comparison
train_fs = train_df['fs']
test_fs = test_df['fs']

plt.figure(figsize=(8,4))
sns.kdeplot(train_fs, label='Train fs', fill=True)
sns.kdeplot(test_fs, label='Test fs', fill=True, color='orange')
plt.title('Distribution Comparison: fs (Train vs Test)')
plt.legend()
plt.show()

# Boxplot for signal length (train)
plt.figure(figsize=(6,4))
sns.boxplot(x=train_df['sig_len'], color='lightblue')
plt.title('Boxplot: Signal Length (Train)')
plt.xlabel('sig_len')
plt.show()

# Correlation heatmap (train)
plt.figure(figsize=(4,3))
sns.heatmap(train_df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap (Train Data)')
plt.show()


## Parquet File (sample_submission.parquet) Analysis

In [ ]:
import pandas as pd

# Load the parquet file
sample_path = '/kaggle/input/physionet-ecg-image-digitization/sample_submission.parquet'
sample_df = pd.read_parquet(sample_path)

# Basic info
print("=== SAMPLE SUBMISSION INFO ===")
print(sample_df.info())

# Preview data
print("\n=== SAMPLE SUBMISSION PREVIEW ===")
display(sample_df.head())

# Unique ID count
print("\nUnique IDs in sample:", sample_df['id'].nunique())

# Describe numeric columns (if any)
print("\n=== Numeric Summary ===")
display(sample_df.describe())


##  EDA Summary — PhysioNet ECG Image Digitization Dataset

### 🔹 Dataset Overview
- **Train set:** 977 samples  
- **Test set:** 24 samples  
- **Sample submission:** 1 Parquet file (`sample_submission.parquet`)  
- No missing values found in any dataset ✅  
- All numeric columns are clean (`id`, `fs`, `sig_len`)

---

### 🔹 Feature Insights
| Feature | Meaning | Type | Observation |
|----------|----------|------|--------------|
| id | Unique ECG record ID | Integer | Unique per sample |
| fs | Sampling frequency | Integer | Ranges from ~250 to 1025 Hz |
| sig_len | Signal length | Integer | Roughly proportional to `fs` (≈ ×10) |

---

### 🔹 Train vs Test Comparison
- Both **train** and **test** have similar `fs` distributions → good consistency.  
- `sig_len` in train data shows no extreme outliers.  
- Test data includes an additional `lead` column (ECG lead type).  

---

### 🔹 Correlation
- Strong positive correlation between `fs` and `sig_len` (~1.0).  
- Indicates each signal’s length depends on its sampling frequency.  

---

### 🔹 Sample Submission File
- The Parquet file contains the **submission structure** — likely `id` + prediction columns.  
- Useful for building final prediction output.

---

### Key Takeaways
- Dataset is **clean, well-structured, and balanced**.  
- Feature engineering can focus on relationships between `fs`, `sig_len`, and ECG images.  
- No preprocessing required for missing/null handling.

---
 **Next Step:**  
Move on to **ECG Image Reconstruction / Signal Extraction** from the `train/` directory,  
then visualize and process the raw ECG waveforms.


## ECG Image File List & Size Check

In [ ]:
import os
from PIL import Image
import random

TRAIN_DIR = '/kaggle/input/physionet-ecg-image-digitization/train'
TEST_DIR = '/kaggle/input/physionet-ecg-image-digitization/test'

# Recursively find all .png files in train dir
train_files = []
for root, dirs, files in os.walk(TRAIN_DIR):
    for f in files:
        if f.endswith('.png'):
            train_files.append(os.path.join(root, f))

# Recursively find all .png files in test dir
test_files = []
for root, dirs, files in os.walk(TEST_DIR):
    for f in files:
        if f.endswith('.png'):
            test_files.append(os.path.join(root, f))

print("Total train images:", len(train_files))
print("Total test images:", len(test_files))

# Check a random train image (if exists)
if train_files:
    sample_img = random.choice(train_files)
    with Image.open(sample_img) as img:
        print("Sample image path:", sample_img)
        print("Image size:", img.size, "Mode:", img.mode)
        display(img)
else:
    print("No train images found. Only test images available.")


## Grayscale Conversion & Pixel-to-Signal Extraction

In [ ]:
import pandas as pd

# Function to convert image to 1D signal
def image_to_signal(img_path):
    with Image.open(img_path) as img:
        gray = img.convert("L")
        arr = np.array(gray)
        signal = arr.mean(axis=0)  # collapse vertical axis
        signal = (signal - signal.min()) / (signal.max() - signal.min())  # normalize
    return signal

# Extract signals for first 5 train images (or fewer if not enough)
num_samples = min(5, len(train_files))
signals = []

for i in range(num_samples):
    sig = image_to_signal(train_files[i])
    signals.append(sig)

# Convert to DataFrame (each row = one image's signal)
max_len = max(len(s) for s in signals)
signal_df = pd.DataFrame([np.pad(s, (0, max_len - len(s)), 'constant') for s in signals])

print("Signal DataFrame shape:", signal_df.shape)
display(signal_df.head())

# Plot all signals
plt.figure(figsize=(12,6))
for i, s in enumerate(signals):
    plt.plot(np.arange(len(s)), s, label=f"Image {i+1}")
plt.title("ECG Signals from Images")
plt.xlabel("Time (pixels)")
plt.ylabel("Normalized Amplitude")
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Make processed folder
processed_dir = '/kaggle/working/processed'
os.makedirs(processed_dir, exist_ok=True)

# Extract signals for test images
test_signals = []
for f in test_files:
    sig = image_to_signal(f)
    test_signals.append(sig)

# Convert to DataFrame
max_len = max(len(s) for s in test_signals)
test_signal_df = pd.DataFrame([np.pad(s, (0, max_len - len(s)), 'constant') for s in test_signals])

# Save CSV
test_csv_path = f'{processed_dir}/test_signals.csv'
test_signal_df.to_csv(test_csv_path, index=False)
print("Test signals saved to:", test_csv_path)

# Quick EDA on test signals
print("Test signals shape:", test_signal_df.shape)

# Plot all test signals
plt.figure(figsize=(12,6))
for i, s in enumerate(test_signals):
    plt.plot(np.arange(len(s)), s, label=f'Signal {i+1}')
plt.title("Reconstructed ECG Signals (Test)")
plt.xlabel("Time (pixels)")
plt.ylabel("Normalized Amplitude")
plt.legend()
plt.show()


## Pretrained Model

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input

# Input shape (for image) 
input_shape = (224,224,3)  # signal -> spectrogram if needed

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  # binary classification example

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


## Prepare Small Dataset

In [ ]:
import os

TRAIN_DIR = '/kaggle/input/physionet-ecg-image-digitization/train'

image_files = []
for root, dirs, files in os.walk(TRAIN_DIR):
    for file in files:
        if file.endswith('.png'):
            # store relative path from TRAIN_DIR
            rel_dir = os.path.relpath(root, TRAIN_DIR)
            rel_file = os.path.join(rel_dir, file)
            image_files.append(rel_file)

print(f"Total images found: {len(image_files)}")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assign dummy labels (0 or 1) just for demonstration
labels = [0 if i % 2 == 0 else 1 for i in range(len(image_files))]

# Create DataFrame
df = pd.DataFrame({
    'filename': image_files,
    'class': labels
})

# Split into train/validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])

print("Train samples:", len(train_df))
print("Validation samples:", len(val_df))


In [ ]:
# Convert labels to string
df['class'] = df['class'].astype(str)

# Split into train/validation again
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])

print("Train samples:", len(train_df))
print("Validation samples:", len(val_df))


In [ ]:
train_gen = datagen.flow_from_dataframe(
    train_df,
    directory=TRAIN_DIR,
    x_col='filename',
    y_col='class',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

val_gen = datagen.flow_from_dataframe(
    val_df,
    directory=TRAIN_DIR,
    x_col='filename',
    y_col='class',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)


## Build Pretrained Model (Transfer Learning)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam

# Input shape (image size)
input_shape = (224, 224, 3)

# Load ResNet50 without top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)  # binary classification

# Final model
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


## Train the Model

In [ ]:
# Reduce image size
train_gen = datagen.flow_from_dataframe(
    train_df,
    directory=TRAIN_DIR,
    x_col='filename',
    y_col='class',
    target_size=(128,128),
    batch_size=16,
    class_mode='binary'
)
val_gen = datagen.flow_from_dataframe(
    val_df,
    directory=TRAIN_DIR,
    x_col='filename',
    y_col='class',
    target_size=(128,128),
    batch_size=16,
    class_mode='binary'
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Stop early if val_loss doesn't improve
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stop]
)
